<a href="https://colab.research.google.com/github/PierreKimbanziR/pneumonia-challenge/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import keras
from keras.models import Model
from keras.layers import Dense
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [4]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/content/drive/MyDrive/pneumonia_challenge/chest_xray/train",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/content/drive/MyDrive/pneumonia_challenge/chest_xray/test", target_size=(224,224))

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [2]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

553476096/553467096 [==============================] - 5s 0us/step


In [5]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

In [9]:
X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(inputs = vggmodel.input, outputs = predictions)

In [13]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(learning_rate=0.0001, momentum=0.9), metrics=["accuracy"])

In [14]:
model_final.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("/content/drive/MyDrive/pneumonia_challenge/vgg16_1.h5", monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)
early = EarlyStopping(monitor='loss', min_delta=0, patience=25, verbose=1, mode='auto')
model_final.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 100, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])
model_final.save_weights("/content/drive/MyDrive/pneumonia_challenge/vgg16_1.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
1/2 [==============>...............] - ETA: 6s - loss: 0.0507 - accuracy: 0.9688
Epoch 00001: loss improved from inf to 0.05068, saving model to /content/drive/MyDrive/pneumonia_challenge/vgg16_1.h5
2/2 [==============================] - ETA: 0s - loss: 0.0728 - accuracy: 0.9688
Epoch 00001: loss did not improve from 0.05068
2/2 [==============================] - 22s 15s/step - loss: 0.0728 - accuracy: 0.9688 - val_loss: 0.5808 - val_accuracy: 0.8438
Epoch 2/100
1/2 [==============>...............] - ETA: 13s - loss: 0.0119 - accuracy: 1.0000
Epoch 00002: loss improved from 0.05068 to 0.01188, saving model to /content/drive/MyDrive/pneumonia_challenge/vgg16_1.h5
2/2 [==============================] - ETA: 0s - loss: 0.0250 - accuracy: 0.9844 
Epoch 00002: loss did not improve from 0.01188
2/2 [==============================] - 20s 6s/step - loss: 0.0250 - accuracy: 0.9844 - val_loss: 0.2799 - val_accuracy: 0.9062
Epoch 3/100
1/2 [==============>...............] - ETA: 6s - 